## Imports

In [ ]:
from pathlib import Path

import h5py
import numpy as np
import scipy as sp
import skimage as ski
import matplotlib.pyplot as plt
from ipywidgets import interact

from fusion import optimize
from utils import hyperspectral2RGB, hyperspectral2RGBvolume, bin_data
from utils import RESOURCES_PATH

RESOURCES_PATH = Path("/Users/federicosimoni/Library/Mobile Documents/com~apple~CloudDocs/Università/Tesi/Code/CS-FLIM_lab/20240703/kidney")

CMOS_PATH = RESOURCES_PATH / "kidney2_520_4m_rec_FOVcorrected.mat"
SPC_PATH = RESOURCES_PATH / "520_kidney2_550_550_610_SPC_raw_proc_tlxy.mat"
LAMBDA_PATH = RESOURCES_PATH / "610_Lambda_L16.mat"

# CMOS_PATH = RESOURCES_PATH / "kidney_cells_520_610_w4_rec_Hil2D_FOVcorrected.mat"
# SPC_PATH = RESOURCES_PATH / "520_kidneyCells_550_550_610_SPC_raw_proc_tlxy.mat"
# LAMBDA_PATH = RESOURCES_PATH / "610_Lambda_L16.mat"

XY_DIM = 128
Z_START = 5
Z_END = 20

## Data loading

In [ ]:
with h5py.File(CMOS_PATH, "r") as f:
    cmos = np.array(h5py.File(CMOS_PATH)["I"])

if cmos.ndim == 3:
    cmos = np.transpose(cmos, (1, 2, 0))
    cmos = ski.transform.resize(cmos, (XY_DIM, XY_DIM, cmos.shape[2]))
    cmos = np.transpose(cmos, (2, 1, 0))
else:
    cmos = ski.transform.resize(cmos, (XY_DIM, XY_DIM))
    cmos = np.transpose(cmos, (1, 0))

cmos = cmos[Z_START:Z_END]
cmos = cmos / cmos.max()

original_spc = sp.io.loadmat(SPC_PATH)["im"]
original_spc[:, :, 0, 0] = original_spc[:, :, 1, 0]

original_t = np.squeeze(sp.io.loadmat(SPC_PATH)["t"])
lam = np.squeeze(sp.io.loadmat(LAMBDA_PATH)["lambda"])


## Time binning and initial plots

In [ ]:
# cut the time (first)
curve = np.squeeze(np.sum(original_spc, axis=(1, 2, 3)))
val = np.max(curve)
pos_max = np.argmax(curve)
curve = curve[pos_max:]
# b = np.argmin(abs(curve - curve[0] * (1 - np.exp(-1 / 1))))
b = 512

spc = original_spc[pos_max: (pos_max + b)]
t = original_t[pos_max: (pos_max + b)]

# data binning (after cut)
t, spc, dt = bin_data(spc, t, 0.1)
spc = spc / np.max(spc)

time_decay = np.mean(spc, axis=(1, 2, 3))

_, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(t, time_decay)
ax[0].set_title(f"Global time decay - {len(t)} temporal points")
ax[0].set_xlabel("Time [ns]")
ax[0].grid()

ax[1].plot(lam, spc.mean(axis=(0, 2, 3)))
ax[1].set_title(f"Global spectrum - {len(lam)} channels")
ax[1].set_xlabel("Wavelength [nm]")
ax[1].grid()

plt.tight_layout()
plt.show()

In [ ]:
initial_spectrums = np.mean(spc, axis=(0))
min_spectrums, max_spectrums = np.min(initial_spectrums), np.max(initial_spectrums)

initial_times = np.mean(spc, axis=(1))
min_times, max_times = np.min(initial_times), np.max(initial_times)

def spc_spectrum_time_in_a_point(i=0, j=0, z=0):
    _, ax = plt.subplots(1, 4, figsize=(16, 4))
    ax[0].imshow(cmos[z], cmap="gray", vmin=0, vmax=1)
    ax[0].set_title(f"CMOS in {z}")

    ax[1].imshow(hyperspectral2RGB(lam, spc.mean(axis=0)))
    ax[1].scatter([j], [i], c="b")
    ax[1].set_title(f"Colored with spectrum")
    
    ax[2].plot(lam, initial_spectrums[:, i, j])
    ax[2].set_ylim(min_spectrums, max_spectrums)
    ax[2].set_title(f"Spectrum in ({i},{j})")
    ax[2].grid()
    
    ax[3].plot(t, initial_times[:, i, j])
    ax[3].set_ylim(min_times, max_times)
    ax[3].set_title(f"Time in ({i},{j})")
    ax[3].grid()

    plt.tight_layout()
    plt.show()
    
interact(spc_spectrum_time_in_a_point, i=(0, spc.shape[-2] - 1, 1), j=(0, spc.shape[-1] - 1, 1), z=(0, cmos.shape[0] - 1, 1));

## Fusion through optimization

In [ ]:
weights = (1,1,3,1)
x = optimize(spc=spc, cmos=cmos, lr=0.1 ,iterations=60, weights=weights)
x = x.cpu().detach().numpy()

## Visualize results

In [ ]:
slices_rgb = hyperspectral2RGBvolume(lam, np.mean(x, axis=0))

means_spectrums = np.mean(x, axis=(0, 3, 4)).T
min_mean_spectrum = np.min(means_spectrums)
max_mean_spectrum = np.max(means_spectrums)

means_times = np.mean(x, axis=(1, 3, 4)).T
min_mean_times = np.min(means_times)
max_mean_times = np.max(means_times)

zxy = np.sum(x, axis=(0, 1))
zxy /= zxy.max()

def plot_across_z(index_z=0):
    _, ax = plt.subplots(2, 2, figsize=(8, 8))
    ax[0, 0].imshow(zxy[index_z], cmap="gray", vmin=0, vmax=1)
    ax[0, 0].set_title(f"Fused image, z={index_z}")

    ax[0, 1].imshow(slices_rgb[index_z])
    ax[0, 1].set_title(f"Colored with spectrum")

    ax[1, 0].plot(lam, means_spectrums[index_z])
    ax[1, 0].set_ylim(min_mean_spectrum, max_mean_spectrum)
    ax[1, 0].set_title(f"Fused Spectrum, z={index_z}")
    ax[1, 0].grid()

    ax[1, 1].plot(t, means_times[index_z])
    ax[1, 1].set_ylim(min_mean_times, max_mean_times)
    ax[1, 1].set_title(f"Fused Time, z={index_z}")
    ax[1, 1].grid()

    plt.tight_layout()
    plt.show()


interact(plot_across_z, index_z=(0, len(zxy) - 1, 1));

In [ ]:
reconstructed_spectrums = np.mean(x, axis=(0, 2))
min_spectrums, max_spectrums = np.min(reconstructed_spectrums), np.max(reconstructed_spectrums)

reconstructed_times = np.mean(x, axis=(1, 2))
min_times, max_times = np.min(reconstructed_times), np.max(reconstructed_times)

def reconstructed_spectrum_time_in_a_point(i=0, j=0):
    _, ax = plt.subplots(1, 3, figsize=(12, 4))
    z_index = np.argmax(means_spectrums.mean(axis=1))
    ax[0].imshow(slices_rgb[z_index])
    ax[0].scatter([j], [i], c="b")
    ax[0].set_title(f"Colored with spectrum")
    
    ax[1].plot(lam, reconstructed_spectrums[:, i, j])
    ax[1].set_title(f"Spectrum in ({i},{j})")
    ax[1].set_ylim(min_spectrums, max_spectrums)
    ax[1].grid()
    
    ax[2].plot(t, reconstructed_times[:, i, j])
    ax[2].set_title(f"Time in ({i},{j})")
    ax[2].set_ylim(min_times, max_times)
    ax[2].grid()
    
    plt.tight_layout()
    plt.show()
    
interact(reconstructed_spectrum_time_in_a_point, i=(0, x.shape[-2], 1), j=(0, x.shape[-1], 1));

## Save data

In [ ]:
SAVE_PATH = RESOURCES_PATH / "520_kidney2_550_550_610_SPC_raw_proc_tlxy_fused"
np.savez(SAVE_PATH, x=x, t=t, lam=lam, weights=weights)